In [1]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np
from scipy.stats import gaussian_kde
matplotlib.rcParams['animation.embed_limit'] = 2**128

In [2]:
def next_collision_time(positions, velocities, i = None, j = None, without = None):
    if i is None:
        i = 0
        j = len(positions) - 2
    
    if j is None:
        j = i + 1
    else:
        j = j + 1

    time = np.divide(positions[i:j] - positions[i + 1:j + 1], velocities[i + 1:j + 1] - velocities[i:j], out = np.full(j - i, np.inf), where = velocities[i + 1:j + 1] != velocities[i:j])
    time[time < 0] = np.inf
    if without is not None:
        time[without] = np.inf
    min_val = np.min(time)
    return i + np.flatnonzero(time == min_val), min_val

In [3]:
def calculate_collision(velocities, i, j = None, masses = None, v_temp = None):
    if j is None:
        j = i + 1

    if i == 0:
        #return velocities[i], -velocities[j]
        return velocities[i], 2 * velocities[i] - velocities[j]
    
    if j == len(velocities) - 1:
        return (2 * velocities[j] - velocities[i]) if v_temp is None else -v_temp, velocities[j]
        
    if masses is None:
        mi, mj = 1, 1
    
    vi = (velocities[i] * (mi - mj) + 2 * mj * velocities[j]) / (mi + mj)
    vj = (velocities[j] * (mj - mi) + 2 * mi * velocities[i]) / (mi + mj)
    
    return vi, vj

In [4]:
def finalise_collision(positions, velocities, vi, vj, i, j = None):
    if j is None:
        j = i + 1 
    
    velocities[i] = vi
    velocities[j] = vj
    
    if positions[i] > positions[j]:
        positions[i], positions[j] = positions[j], positions[i]

In [5]:
def calculate_positions(positions, velocities, dt, i = None, j = None):
    if i is None:
        i = 0
        j = len(positions) - 1        
    if j is None:
        j = i
    
    positions[i:j + 1] += velocities[i:j + 1] * dt

In [6]:
def next_step(positions, velocities, dt, v_temp = None):
    current_time = 0
    sensor_collision_momentums = []
    without = None
    while(True):
        inds, collision_time = next_collision_time(positions, velocities, without = without)
        if current_time + collision_time > dt:
            calculate_positions(positions, velocities, dt - current_time)
            return np.array(sensor_collision_momentums)
        
        calculate_positions(positions, velocities, collision_time)
        for index in inds:
            vi, vj = calculate_collision(velocities, index, v_temp = v_temp)
            if index == len(positions) - 2:
                sensor_collision_momentums.append(np.abs(vi - velocities[index]))            
            finalise_collision(positions, velocities, vi, vj, index)
        current_time += collision_time
        without = inds

In [54]:
T = 500
print(f"Temperature: {T}")
v = np.sqrt(2 * T)
print(f"Velocity: {v}")

min_x = 0
max_x = 100
print(f"Time: {(max_x - min_x) / v}")
particles = 100
particle_lines = 80
dx = (max_x - min_x) / (particles + 1)

lines = []
for i in range(particle_lines):
    positions = np.linspace(min_x, max_x, particles + 2)
    positions[1:-1] += np.random.uniform(-0.9 * dx / 2, 0.9 * dx / 2, len(positions) - 2)    
    velocities = np.random.choice([-v, v], particles + 2) + np.random.normal(0, 0.25 * v, particles + 2)
    positions[0] = 0
    positions[-1] = 100
    velocities[0] = 0
    velocities[-1] = 0
    lines.append((positions, velocities))

xs = np.linspace(min_x, max_x, 100)
bins = 10

time = 30
fps = 60

sensor_time = 30
plot_time = 10
plot_points = fps * plot_time

plot_xs = np.linspace(0, 1, plot_points)
plot_ys_sensor = np.zeros_like(plot_xs)
plot_ys_piston = np.zeros_like(plot_xs)

piston_w = 0.5
piston_A = 2

Temperature: 500
Velocity: 31.622776601683793
Time: 3.1622776601683795


In [98]:
def piston_velocity_sin(t, w, A):
    if t > 6 and t < 6 + np.pi / w:
        return A * w * np.cos(w * (t - 6))
    return 0

In [8]:
def piston_velocity_sin(t, w, A):
    if t > 6 and t < 6.5:
        return A / 0.5
    if t > 6.5 and t < 7:
        return -A / 0.5    
    return 0

In [55]:
#fig = plt.figure(figsize = (20, 10))
ppi = 300
fig = plt.figure(figsize = (3840 / ppi, 2160 / ppi), dpi = ppi)
gs = GridSpec(2, 2, figure = fig, width_ratios = (2, 1))
ax1 = plt.subplot(gs[:, 0])
ax3 =  plt.subplot(gs[0, 1])
ax4 =  plt.subplot(gs[1, 1])
#, ((, ax3), (ax2, ax4)) = plt.subplots(2, 2, figsize = (20, 10), gridspec_kw={'width_ratios': [2, 1]})

ax1.set_xlim(-5, 105)
ax1.set_ylim(-1, particle_lines)
ax1.set_yticks([])
ax1_lines = [ax1.plot([], [], 'o', lw = 2, ms = 2)[0] for _ in range(particle_lines)]

ax3.set_xlim(0, 1)
ax3_line, = ax3.plot([], [], '-', lw = 2)
ax4.set_xlim(0, 1)
ax4_line, = ax4.plot([], [], '-', lw = 2)

plt.close()

sensor_collision_momentums = [None] * sensor_time

def animate(i):
    global sensor_collision_times, plot_ys_sensor, plot_ys_piston
    
    ax3_line.set_data(plot_xs, plot_ys_piston)

    min_y, max_y = ax3.get_ylim()
    max_y = max_y if max_y > plot_ys_piston[-1] else plot_ys_piston[-1]
    min_y = min_y if min_y < plot_ys_piston[-1] else plot_ys_piston[-1]
    ax3.set_ylim(min_y, max_y)
    ax3.set_xlim(i / fps - plot_time, i / fps)
    ax3_line.set_data(i / fps - plot_time * np.flip(plot_xs), plot_ys_piston)
    
    _, lim_y = ax4.get_ylim()
    lim_y = lim_y if lim_y > plot_ys_sensor[-1] else plot_ys_sensor[-1] * 1.5
    ax4.set_ylim(0, lim_y)
    ax4.set_xlim(i / fps - plot_time, i / fps)
    ax4_line.set_data(i / fps - plot_time * np.flip(plot_xs), plot_ys_sensor)

    
    mts = []
    for j, ((pos, vel), ax1_line) in enumerate(zip(lines, ax1_lines)):
        vel[0] = piston_velocity_sin(i / fps, piston_w, piston_A)
        ax1_line.set_data(pos, np.zeros_like(pos) + j)
        mts.append(next_step(pos, vel, 1 / fps))
    
    plot_ys_sensor = np.roll(plot_ys_sensor, -1)
    scm = [scm for scm in sensor_collision_momentums if scm is not None]
    if len(scm) > 0:
        plot_ys_sensor[-1] = np.sum(np.concatenate(scm)) * fps / sensor_time
    else:
        plot_ys_sensor[-1] = 0
    sensor_collision_momentums[i % sensor_time] = np.concatenate(mts)
    #print(plot_ys_sensor[-1])
    
    plot_ys_piston = np.roll(plot_ys_piston, -1)
    plot_ys_piston[-1] = lines[0][0][0]
    
    return *ax1_lines, ax3_line, ax4_line

anim = animation.FuncAnimation(fig, animate, fps * time, interval=1000 / fps, blit=True)
#anim.save()
#HTML(anim.to_html5_video())


In [56]:
anim.save('sound-animation-p100-t500.mp4', writer = animation.FFMpegWriter(fps = 60))

In [27]:
fig, ((ax1, ax3), (ax2, ax4)) = plt.subplots(2, 2, figsize = (20, 10), gridspec_kw={'width_ratios': [2, 1]})
ax1.set_xlim(-5, 105)
ax1.set_ylim(-1, particle_lines)
ax1.set_yticks([])
ax1_lines = [ax1.plot([], [], 'o', lw = 2, ms = 4)[0] for _ in range(particle_lines)]

ax3.set_xlim(0, 1)
ax3_line, = ax3.plot([], [], '-', lw = 3)
ax4.set_xlim(0, 1)
ax4_line, = ax4.plot([], [], '-', lw = 3)

plt.close()

sensor_collision_momentums = [None] * sensor_time

def animate(i):
    global sensor_collision_times, plot_ys_sensor, plot_ys_piston
    
    joined_positions = np.concatenate([pos[1:-1] for pos, vel in lines])
    ax2.cla()
    ax2.set_ylim(0, 0.02)
    ax2.set_xlim(-5, 105)
    ax2.grid(ls = '--')
    ax2.hist(joined_positions, density = True, color = (0.9, 0.9, 0.9), bins = np.linspace(min_x, max_x, bins + 2))
    kernel = gaussian_kde(joined_positions)
    ax2.plot(xs, kernel.evaluate(xs), lw = 3, color = 'C3')    
    
    ax3_line.set_data(plot_xs, plot_ys_piston)

    min_y, max_y = ax3.get_ylim()
    max_y = max_y if max_y > plot_ys_piston[-1] else plot_ys_piston[-1]
    min_y = min_y if min_y < plot_ys_piston[-1] else plot_ys_piston[-1]
    ax3.set_ylim(min_y, max_y)
    ax3_line.set_data(plot_xs, plot_ys_piston)
    
    _, lim_y = ax4.get_ylim()
    lim_y = lim_y if lim_y > plot_ys_sensor[-1] else plot_ys_sensor[-1] * 1.2
    ax4.set_ylim(0, lim_y)
    ax4_line.set_data(plot_xs, plot_ys_sensor)

    
    mts = []
    for j, ((pos, vel), ax1_line) in enumerate(zip(lines, ax1_lines)):
        vel[0] = piston_velocity_sin(i / fps, piston_w, piston_A)
        ax1_line.set_data(pos, np.zeros_like(pos) + j)
        mts.append(next_step(pos, vel, 1 / fps))
    
    plot_ys_sensor = np.roll(plot_ys_sensor, -1)
    scm = [scm for scm in sensor_collision_momentums if scm is not None]
    if len(scm) > 0:
        plot_ys_sensor[-1] = np.sum(np.concatenate(scm)) * fps / sensor_time
    else:
        plot_ys_sensor[-1] = 0
    sensor_collision_momentums[i % sensor_time] = np.concatenate(mts)
    #print(plot_ys_sensor[-1])
    
    plot_ys_piston = np.roll(plot_ys_piston, -1)
    plot_ys_piston[-1] = lines[0][0][0]
    
    return *ax1_lines, ax3_line, ax4_line

anim = animation.FuncAnimation(fig, animate, fps * time, interval=1000 / fps, blit=True)
HTML(anim.to_html5_video())

KeyboardInterrupt: 

In [10]:
x = [1, 2, 3, None, 4]
[filter(None, x)]